In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [8]:
dataSetOne = pd.read_csv('prep.csv', index_col= None)
dataSetTwo = dataSetOne
dataSetTwo = pd.get_dummies(dataSetTwo, drop_first= True)

indep_X = dataSetTwo.drop(['classification_yes'], axis= 1)
dep_Y = dataSetTwo['classification_yes']

In [3]:
def SelectBest(indep_X, dep_Y, n):
    test = SelectKBest(score_func= chi2, k = n)
    fit1 = test.fit(indep_X, dep_Y)

    # what are the columns are taken from the selectKBest
    columns  = fit1.get_support(indices= True)
    featureDataSetColumns = dataSetTwo.iloc[:, columns]
   # what are the columns are taken from the selectKBest
    
    SelectK_Feature = fit1.transform(indep_X)
    return SelectK_Feature, featureDataSetColumns

In [9]:
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, y_train, y_test

In [22]:
def cm_prediction(classifier,X_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score 
    from sklearn.metrics import classification_report 
    
    
    cm = confusion_matrix(y_test, y_pred)
    Accuracy=accuracy_score(y_test, y_pred )
    report=classification_report(y_test, y_pred)
    
    return  classifier,Accuracy,report,X_test,y_test,cm

In [23]:
# algorithms Logistic, svm_linear, svm_NL, navie bayes, knn, decision, random
def Logistic(X_train, y_train, X_test):
    start = time.time()
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state= 0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm= cm_prediction(classifier, X_test)
    end = time.time()
    timing = end - start
    return classifier, Accuracy, report, X_test, y_test, cm, timing

def svm_linear(X_train,y_train,X_test):
        start = time.time()      
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing

def svm_NL(X_train,y_train,X_test):
        start = time.time()         
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing

def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        start = time.time()
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        start = time.time()
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        start = time.time()
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        start = time.time()
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        end = time.time()
        timing = end - start
        return  classifier, Accuracy, report, X_test, y_test, cm, timing

In [26]:
# out put data frame
def selectk_Classification(acclog, accsvml, accsvmnl, accnav, accknn, accdes, accrf): 
    accuracyAndTiming = [acclog, accsvml, accsvmnl, accnav, accknn, accdes, accrf]
    print(accuracyAndTiming)
    dataframe=pd.DataFrame(index=['Logistic', 'SVMl', 'SVMnl', 'Navie bayes', 'Knn', 'Decision Tree', 'Random Forest'],columns=['ChiSquare','timing'])
    for iterOne, index in enumerate(dataframe.index):
        s = accuracyAndTiming[iterOne]
        for iterTwo, columns in enumerate(dataframe.columns):
              dataframe[columns][index] = s[0][iterTwo]
    return dataframe

In [28]:
# load accuracy values
acclog = []
accsvml = []
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [29]:
# select the best of n number
kBest, featureDataSetColumns = SelectBest(indep_X, dep_Y, 5)

# split X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_scalar(indep_X, dep_Y)

# algorithm
classifier, Accuracy,report, X_test, y_test, cm, timing= Logistic(X_train,y_train,X_test)
acclog.append([Accuracy, timing])

classifier, Accuracy, report, X_test, y_test, cm, timing=svm_linear(X_train,y_train,X_test)  
accsvml.append([Accuracy, timing])

classifier, Accuracy, report, X_test, y_test, cm, timing= svm_NL(X_train,y_train,X_test)  
accsvmnl.append([Accuracy, timing])

classifier, Accuracy,report, X_test, y_test, cm, timing= Navie(X_train,y_train,X_test)
accnav.append([Accuracy, timing])

classifier, Accuracy, report, X_test, y_test, cm, timing= knn(X_train,y_train,X_test)  
accknn.append([Accuracy, timing])

classifier, Accuracy, report, X_test, y_test, cm, timing= Decision(X_train,y_train,X_test)  
accdes.append([Accuracy, timing])

classifier, Accuracy, report, X_test, y_test, cm, timing= random(X_train,y_train,X_test)  
accrf.append([Accuracy, timing])
# algorithm

# result
result=selectk_Classification(acclog, accsvml, accsvmnl, accnav, accknn, accdes, accrf)

[[[0.99, 0.01250600814819336]], [[0.98, 0.006796121597290039]], [[0.99, 0.0049512386322021484]], [[0.98, 0.0062143802642822266]], [[0.97, 0.012212276458740234]], [[0.9, 0.042462825775146484]], [[0.99, 0.03613853454589844]]]


C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\REVANTH KUMAR\Anaconda3\envs\hopeAi\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\REVANTH KUMAR\Anaconda3

In [30]:
result

,ChiSquare,timing
Logistic,0.99,0.012506
SVMl,0.98,0.00679612
SVMnl,0.99,0.00495124
Navie bayes,0.98,0.00621438
Knn,0.97,0.0122123
Decision Tree,0.9,0.0424628
Random Forest,0.99,0.0361385
